In [212]:
import numpy as np
import pandas as pd
from elasticsearch import Elasticsearch
from sklearn.model_selection import train_test_split
import geopandas as gpd

In [213]:

HOST = 'elasticsearch-master.projet-ssplab'

def elastic():
    """Connection avec Elastic sur le data lab"""
    es = Elasticsearch([{'host': HOST, 'port': 9200, 'scheme': 'http'}], http_compress=True, request_timeout=200)
    return es

es = elastic()

In [214]:
# Importation des bases
import functions as fc
dict_data = fc.read_all_raw(fc.list_bases)
dict_data.keys()

dict_keys(['rejets', 'etablissements', 'emissions', 'Trait_dechets_non_dangereux', 'Trait_dechets_dangereux', 'Prod_dechets_non_dangereux', 'Prod_dechets_dangereux', 'Prelevements'])

In [235]:
 dict_data["etablissements"].columns

Index(['identifiant', 'nom_etablissement', 'numero_siret', 'adresse',
       'code_postal', 'commune', 'departement', 'region', 'coordonnees_x',
       'coordonnees_y', 'code_epsg', 'code_ape', 'libelle_ape', 'code_eprtr',
       'libelle_eprtr'],
      dtype='object')

In [215]:
df = dict_data["etablissements"]
df = df.rename({'numero_siret': "numero_siret_true"}, axis = 1)
df["numero_siret_true"] = df["numero_siret_true"].astype(str)

In [216]:
df[["coordonnees_x", "coordonnees_y", "code_epsg"]].head()

,coordonnees_x,coordonnees_y,code_epsg
0,672948.910000,1.813634e+06,27572.0
1,-1.498014,4.350209e+01,4326.0
2,2.856548,5.043594e+01,4326.0
3,2.140590,4.898410e+01,4326.0
4,6.861504,4.762541e+01,4326.0


In [217]:
df["code_epsg"].value_counts()

2154.0     7520
4326.0      882
27572.0     368
4559.0       33
2971.0        8
3727.0        4
Name: code_epsg, dtype: int64

In [218]:
df["code_apet"] = df["code_ape"].str[:4]

In [219]:
etab_not_null = df.dropna(subset = ['code_epsg'])
etab_null = df.loc[df['code_epsg'].isnull()]
gb = etab_not_null.groupby("code_epsg")
gb = [gb.get_group(x) for x in gb.groups]

def transform_wgs84(df, epsg):
    etab = gpd.GeoDataFrame(
       df,
        geometry=gpd.points_from_xy(
            df['coordonnees_x'],
            df['coordonnees_y']
        ),
        crs = epsg)
    etab = etab.to_crs(4326)
    etab['x'] = etab['geometry'].x 
    etab['y'] = etab['geometry'].y
    etab = pd.DataFrame(etab)
    return etab

In [220]:
df.columns

Index(['identifiant', 'nom_etablissement', 'numero_siret_true', 'adresse',
       'code_postal', 'commune', 'departement', 'region', 'coordonnees_x',
       'coordonnees_y', 'code_epsg', 'code_ape', 'libelle_ape', 'code_eprtr',
       'libelle_eprtr', 'code_apet'],
      dtype='object')

In [221]:
temp = [
    transform_wgs84(
        gb[idx],
        gb[idx]['code_epsg'].iloc[0]
    ) for idx in range(len(gb)) 
]
temp2 = pd.concat(
    temp
)
temp3 = pd.concat(
    [temp2, etab_null]
)



In [ ]:
temp3.columns

In [ ]:
temp2.columns

In [ ]:
temp3.shape

In [ ]:
temp3[['x', "y"]].head()

In [222]:
X = temp3

In [223]:
X_train, X_test = train_test_split(
     X, test_size=0.20, random_state=42)

In [224]:
X_train.columns

Index(['identifiant', 'nom_etablissement', 'numero_siret_true', 'adresse',
       'code_postal', 'commune', 'departement', 'region', 'coordonnees_x',
       'coordonnees_y', 'code_epsg', 'code_ape', 'libelle_ape', 'code_eprtr',
       'libelle_eprtr', 'code_apet', 'geometry', 'x', 'y'],
      dtype='object')

In [298]:
requete_type = '''{{ 
  "query": {{
    "bool": {{
      "should": [
        {{"multi_match" : {{"query":"{nom_etablissement}",  "type":"best_fields", "fields": [ "denom", "enseigne", "nom_comm_et", "adr_et_l1","adr_et_l2", "denom_condense", "enseigne_et1" ],"tie_breaker": 0.1}}}},
        {{ "match": {{ "sir_adr_et_com_lib":  "{commune}" }}}}
      ],
      "filter": [
        {{ "prefix":  {{ "apet": "{code_apet}" }}}}
      ]
    }}
  }},
  "size": 1
}}'''

In [267]:
X_train[["nom_etablissement", "commune","code_apet", "code_postal"]].iloc[0,:]

nom_etablissement       DEA VILLACOUBLAY
commune              VELIZY-VILLACOUBLAY
code_apet                           8422
code_postal                        78129
Name: 643, dtype: object

In [299]:
requete_type.format_map({'nom_etablissement': 'DEA VILLACOUBLAY','commune':'VELIZY-VILLACOUBLAY', 'code_apet': '8422'})

'{ \n  "query": {\n    "bool": {\n      "should": [\n        {"multi_match" : {"query":"DEA VILLACOUBLAY",  "type":"best_fields", "fields": [ "denom", "enseigne", "nom_comm_et", "adr_et_l1","adr_et_l2", "denom_condense", "enseigne_et1" ],"tie_breaker": 0.1}},\n        { "match": { "sir_adr_et_com_lib":  "VELIZY-VILLACOUBLAY" }}\n      ],\n      "filter": [\n        { "prefix":  { "apet": "8422" }}\n      ]\n    }\n  },\n  "size": 1\n}'

In [300]:
# Il est nécessaire de spécifier l'index associé à chaque requête
header = '{"index" : "sirus_2020"}'

multiple_requetes = ""

# On itère sur le dataframe d'établissements polluants pour ajouter une requête spécifique à chacun d'entre eux
n_etab = X_train.shape[0] # Pour l'exemple, on prend les 10 premiers

for index, row in X_train.iloc[0:n_etab][['nom_etablissement','commune', 'code_apet']].iterrows():
    
    multiple_requetes+= header
    multiple_requetes+= '\n'
    multiple_requetes+= requete_type.format_map(row).replace("\n","")
    multiple_requetes+= '\n'

In [301]:
res = es.msearch(body = multiple_requetes)

/home/coder/local/bin/conda/envs/basesspcloud/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [302]:
len(res['responses'])

7170

In [303]:
res['responses']

[{'took': 16,
  'timed_out': False,
  '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0},
  'hits': {'total': {'value': 3, 'relation': 'eq'},
   'max_score': 0.0,
   'hits': [{'_index': 'sirus_2020_e_3_ngr_bool',
     '_type': '_doc',
     '_id': '18009006002712',
     '_score': 0.0,
     '_source': {'sirus_id': '180090060',
      'nic': '02712',
      'ape': '8899B',
      'apet': '8422Z',
      'eff_3112_et': '0.0',
      'eff_etp_et': '0.0',
      'eff_et_effet_daaaammjj': '20161231',
      'enseigne_et1': 'IGESA CI PEDESTRE',
      'nom_comm_et': '',
      'adr_et_loc_geo': '3328100297',
      'adr_et_compl': 'ART AQUITAINE - BASE AERIENNE 106',
      'adr_et_voie_num': '',
      'adr_et_voie_repet': '',
      'adr_et_voie_type': 'AV',
      'adr_et_voie_lib': 'DE L ARGONNE',
      'adr_et_cedex': '33693',
      'adr_et_distsp': 'CS 70037',
      'sir_adr_et_com_lib': 'MERIGNAC CEDEX',
      'adr_et_post': '33700',
      'adr_et_l1': 'INSTITUTION GESTION SOCIALE DE

In [304]:
X_train["siret_elastic"] = [res['responses'][i]['hits']['hits'][0]["_source"]["siret_id"] if \
    res['responses'][i]['hits']['hits'] else np.NaN for i in range(X_train.shape[0]) ]

In [305]:
X_train[["siret_elastic", "numero_siret_true"]].head(20)

,siret_elastic,numero_siret_true
643,18009006002712,15100003100741
2471,53540643300024,43361050800031
3072,33898588000032,33898588000032
6250,32623391300176,32623391300028
5451,33470873200105,33470873200105
6648,73820183900011,73820183900029
6371,79361882800024,79361882800024
3748,38034280800132,38034280800132
8772,32539647100030,32539647100030
1670,34229657100025,34229657100025


In [306]:
X_train["match"] = (X_train["numero_siret_true"] == X_train["siret_elastic"])

In [307]:
X_train["match"].value_counts()/X_train.shape[0] # 55.6%

True     0.556067
False    0.443933
Name: match, dtype: float64

In [308]:
X_train["siret_elastic"].isna().sum()

0